In [43]:
import glob
import slab 
from slab import dataanalysis
import numpy as np
import subprocess
import time
import os
from h5py import File, special_dtype
import threading
import gc
import yaml
from datetime import datetime, timedelta

In [37]:
clear

In [57]:
class Server():
    def __init__(self, config_path=None, launch_at_start=True):
        if config_path==None:
            local_path=os.getcwd()
            config_path=local_path+'\\furnace_config.yml'
        try:
            with open(config_path, 'r') as f:
                self.RGA_config=yaml.full_load(f)['RGA']
            self.data_path = self.RGA_config['data_path']
            if self.data_path==None:
                log_dir='\\Furnace Logs'
                self.data_path=os.path.dirname(config_path)+log_dir
                if not os.path.exists(self.data_path):
                    os.makedirs(self.data_path)
                self.data_path=self.data_path+'\\'
            
        except:
            print('Unable to find or open server, check config_path and config.')
        if launch_at_start:
            self.log, self.PID = self.pressure_server(self.data_path)
        else:
            pass

    def get_log(self):
        return self.log

    def get_PID(self):
        return self.PID

    def last_log_fname(self, path, prefix):
        dirlist = glob.glob(path + '*' + prefix + '*')
        dsort = dirlist.sort()
        i = 1
        max_iter = 100
        if dsort == None and len(dirlist) == 0:
            date = prefix.split('_')[0]
            while dsort == None and len(dirlist) == 0 and i < max_iter:
                prefix = (datetime.strptime(date, '%Y-%m-%d') - timedelta(days=i)).strftime('%Y-%m-%d') + '_RGA_log'
                dirlist = glob.glob(path + '*' + prefix + '*')
                dirlist.sort()
                i += 1
            if i == max_iter:
                return None
            else:
                f_name = os.path.split(dirlist[-1])[-1]
        else:
            f_name = os.path.split(dirlist[-1])[-1]
        return f_name

    def pressure_server(self, data_path):
        next_fname = self.next_log_fname(data_path)
        num = next_fname.split('_')[0]
        prefix = next_fname.split(num + '_')[1]
        last_fname = self.last_log_fname(data_path, prefix)
        if last_fname!=None:
            log_time = self.swmr_read(data_path + last_fname, 'tpts', -1)
            et = time.time() - log_time
            if et > 30:
                self.p_log, self.p_log_ID = self.start_pressure_server(data_path)
            else:
                try:
                    PID = self.swmr_attr(data_path + last_fname, 'LOG_PID')
                    p_name = self.process_name(PID)
                    if p_name == 'python.exe':
                        self.p_log = last_fname
                        self.p_log_ID = PID
                        print('Server Running:')
                        print('_________________')
                        print('Server Log: %s' % self.p_log)
                        print('Server Process ID %i' % self.p_log_ID)
                    else:
                        self.p_log, self.p_log_ID = self.start_pressure_server(data_path)
                except:
                    print('ERROR in determining PID from log')
        else:
            self.p_log, self.p_log_ID = self.start_pressure_server(data_path)
        return self.p_log, self.p_log_ID

    def server_state(self):
        p_name = self.process_name(self.PID)
        if p_name == 'python.exe':
            state = True
        else:
            state = False
        return state

    def read(self, key):
        data = self.swmr_read(self.data_path + self.log, key, -1)
        return data

    def attr_read(self, key):
        data=self.swmr_attr(self.data_path+self.log, key)
        return data

    def pp_keys(self):
        data_file = File(self.data_path + self.log, 'r', libver='latest', swmr=True)
        try:
            keys = [key for key in data_file.keys()]
            pp_key_vals = []
            for vals in keys:
                if vals != 'tpts' and vals != 'p_tot':
                    pp_key_vals.append(vals)
                else:
                    pass
            data_file.close()
            return pp_key_vals
        except:
            self.exit_handler()
            raise Exception('ERROR')

    def start_pressure_server(self, data_path, suffix='h5'):
        '''
        Starts an external pressure datalogger using shell command
        '''
        try:
            conf = subprocess.call('start python pressure_server.py', shell=True)
            if conf != 0:
                raise Exception('Error in shell script, check instrument')
            else:
                pass

            new_log_fname = self.next_log_fname(data_path)

            i = 0
            max_iter = 60
            print('Waiting for server to start...')

            while os.path.isfile(data_path + new_log_fname + '.' + suffix) == False and i < max_iter:
                time.sleep(1)
                i += 1
            if i >= max_iter:
                raise Exception('ERROR: Timed out waiting for log to be created')
            else:
                pass
            print('_________________')

            try:
                time.sleep(2)
                PID = self.swmr_attr(data_path + new_log_fname + '.' + suffix, 'LOG_PID')
                print('New log created: %s' % new_log_fname + '.' + suffix)
                print('Server Process ID: %i' % PID)
                return new_log_fname + '.' + suffix, PID
            except:
                raise Exception('ERROR: Server has no PID')

        except RuntimeError:
            print('Error occurred in server initialization')

    def exit_handler(self):
        '''
        Function that finds any instances of open H5 files and safely
        closes them in event of normal interpreter shutdown or error.
        '''
        for obj in gc.get_objects():  # Browse through ALL objects
            if isinstance(obj, File):  # Just HDF5 files
                try:
                    obj.close()
                except:
                    pass  # Was already closed

    def latest_file(self, filepath):
        list_files = glob.glob(filepath + '*')
        latest_file = max(list_files, key=os.path.getctime)
        data_file = latest_file.split('\\')[-1]
        return data_file

    def swmr_read(self, file_path, key, index):
        data_file = File(file_path, 'r', libver='latest', swmr=True)
        try:
            data = np.array(data_file[key])[index]
            data_file.close()
            return data
        except:
            self.exit_handler()

    def swmr_attr(self, file_path, key):
        data_file = File(file_path, 'r', libver='latest', swmr=True)
        try:
            attr_val = data_file.attrs[key]
            data_file.close()
            return attr_val
        except:
            self.exit_handler()
            print('ERROR')

    def kill(self):
        self.kill_server(self.PID)

    def process_name(self, PID):
        '''
        Finds process name for a given process ID using shell commands and parsing return.
        '''
        out = subprocess.check_output('tasklist /fi "pid eq {0}"'.format(PID), shell=True).decode("utf-8")
        if out.split(':')[0] == 'INFO':
            return 'no task'
        else:
            return out.split('\r\n')[3].split(' ')[0]

    def kill_server(self, PID):
        '''
        safely kills the data server via shell. Checks if process is a valid python executable.
        '''
        try:
            p_name = self.process_name(PID)
            if p_name == 'python.exe':
                # needs /t flag for safe process shutdown
                subprocess.Popen('taskkill /PID {0} /t'.format(PID), shell=True)
            elif p_name == 'no task':
                print('No process currently running with that ID')
            else:
                print('Not a valid python interpreter, cannot close process')
        except:
            print('ERROR Occurred in killing process')

    def _get_next_filename(self, datapath,prefix,suffix=''):
        ii = self._next_file_index(datapath, prefix)
        return "%05d_" % (ii) + prefix +suffix

    def _next_file_index(self, datapath,prefix=''):
        """Searches directories for files of the form *_prefix* and returns next number
            in the series"""

        dirlist=glob.glob(os.path.join(datapath,'*_'+prefix+'*'))
        dirlist.sort()
        try:
            ii=int(os.path.split(dirlist[-1])[-1].split('_')[0])+1
        except:
            ii=0
        return ii

    def next_log_fname(self, path):
        local_time = time.time()
        date = datetime.fromtimestamp(local_time).strftime('%Y-%m-%d %H:%M:%S').split(' ')[0]
        prefix = date.split(' ')[0] + '_RGA_log'
        next_file = self._get_next_filename(path, prefix)
        return next_file

In [60]:
server=Server()

Server Running:
_________________
Server Log: 00003_2020-11-30_RGA_log.h5
Server Process ID 20292


In [42]:
server.data_path

'c:\\Users\\Molybdenum\\Documents\\Furnace Automation\\Furnace Logs'

In [12]:
def pressure_server(self, data_path):
        next_fname = self.next_log_fname(data_path)
        num = next_fname.split('_')[0]
        prefix = next_fname.split(num + '_')[1]
        print(prefix)
        last_fname = self.last_log_fname(data_path, prefix)
        print(next_fname, data_path)
        if last_fname!=None:
            log_time = self.swmr_read(data_path + last_fname, 'tpts', -1)
            et = time.time() - log_time
            if et > 30:
                p_log, p_log_ID = self.start_pressure_server(data_path)
                return p_log, p_log_ID
            else:
                try:
                    PID = self.swmr_attr(data_path + last_fname, 'LOG_PID')
                    p_name = self.process_name(PID)
                    if p_name == 'python.exe':
                        p_log = last_fname
                        p_log_ID = PID
                        print('Server Running:')
                        print('_________________')
                        print('Server Log: %s' % p_log)
                        print('Server Process ID %i' % p_log_ID)
                    else:
                        p_log, p_log_ID = self.start_pressure_server(data_path)
                    return p_log, p_log_ID
                except:
                    print('ERROR in determining PID from log')
        else:
            p_log, p_log_ID = self.start_pressure_server(data_path)
            return(p_log, p_log_ID)

In [13]:
pressure_server(server, path)

2020-11-30_RGA_log
00000_2020-11-30_RGA_log C:\Users\Molybdenum\Documents\Furnace Logs\
1.5478551387786865
Server Running:
_________________
Server Log: 00000_2020-11-29_RGA_log.h5
Server Process ID 10612


('00000_2020-11-29_RGA_log.h5', 10612)

In [20]:
#RGA parameter keys
RGA_keys=['fil_current','e_energy','ion_energy','foc_volts','scan_rate','units','t_wait']
config_path='C:\\Users\\Molybdenum\\Documents\\Furnace Automation\\'
config_name='furnace_config.yml'
system_config='system_config.yml'

with open(config_path+config_name, 'r') as f:
    rga_config=yaml.full_load(f)['RGA']

In [22]:
rga_config['data_path']==None

True

In [27]:
glob.glob(config_path+'Furnace Logs')

['C:\\Users\\Molybdenum\\Documents\\Furnace Automation\\Furnace Logs']

In [32]:
log_dir='Furnace Log'
if not os.path.exists(config_path+log_dir):
    os.makedirs(config_path+log_dir)
    print('hey')

hey


In [34]:
os.getcwd()

'c:\\Users\\Molybdenum\\Documents\\Furnace Automation'